all notebooks before I did are based off 5 seconds long clips. However, I cannot get good results from that.

I want to adjust the data process from 5 sec to 30 sec long to see if I could get a better result

In [273]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import random
from typing import List
from torchaudio.transforms import MelSpectrogram, AmplitudeToDB

import torchaudio

import torch

from torch.utils.data import DataLoader,TensorDataset

import lightning as L

import datasets

from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler

from pathlib import Path
import multiprocessing
import colorednoise as cn
import torch.nn as nn
import librosa
from torch.distributions import Beta
from torch_audiomentations import Compose, PitchShift, Shift, OneOf, AddColoredNoise

import timm
from torchinfo import summary

import torch.nn.functional as F

from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts,
    ReduceLROnPlateau,
    OneCycleLR,
)
from lightning.pytorch.callbacks  import ModelCheckpoint, EarlyStopping

from lightning.pytorch.loggers import MLFlowLogger

In [274]:
import gc

In [275]:
path='../../data/train/asiope1/XC194954.ogg'

In [276]:
def read_audio(path: str):
    """
    Read an OGG file using torchaudio and return the waveform tensor and sample rate.

    Parameters:
        path: Path to the .ogg file

    Returns:
        waveform: Tensor representing the waveform
        sample_rate: Sample rate of the audio file
    """
    audio, sample_rate = torchaudio.load(path)
    return audio, sample_rate

In [277]:
audio,sr=read_audio(path=path)

In [278]:
print(audio)
print(audio.shape)

tensor([[-3.4184e-05, -1.9732e-05, -6.3533e-06,  ...,  3.2216e-05,
         -8.1519e-06, -8.0238e-06]])
torch.Size([1, 457247])


In [279]:
def slice_audio_30_align(audio: torch.Tensor, sample_rate: int) -> List[torch.Tensor]:
    """
    Slice the complete audio tensor into multiple 30 seconds length,
    keep all slices having the same length, especially for the last slice.

    Parameters:
        audio: Tensor representing the waveform of the audio.
        sample_rate: The sample rate of the audio file.

    Returns:
        segments: List of tensors, each representing a 30-second audio segment.
    """
    # Set up the segment duration in samples
    segment_duration = 5 * sample_rate  # 5 seconds in number of samples

    # Total number of samples in the waveform
    total_samples = audio.shape[1]

    # Check if the audio is less than 5 seconds
    if total_samples < segment_duration:
        # Calculate the required padding length
        padding_length = segment_duration - total_samples
        # Create a tensor of zeros (silence) for padding
        silence = torch.zeros(audio.shape[0], padding_length)
        # Pad the waveform with silence
        padded_waveform = torch.cat([audio, silence], dim=1)
        return [padded_waveform]  # Return the padded waveform as a single segment

    # If the audio is 5 seconds or longer, proceed as normal
    segments = [audio[:, i:i + segment_duration] for i in range(0, total_samples, segment_duration)]

    # Ensure the last segment is exactly 5 seconds long
    if segments[-1].shape[1] != segment_duration:
        # Extract the last segment_duration samples to ensure it's 5 seconds long
        last_segment = audio[:, -segment_duration:]
        segments[-1] = last_segment  # Replace the last segment with a full 5-second segment

    return segments

In [280]:
segments=slice_audio_30_align(audio=audio,sample_rate=sr)

In [281]:
print(segments)
print(segments[0].shape)

[tensor([[-3.4184e-05, -1.9732e-05, -6.3533e-06,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])]
torch.Size([1, 960000])


In [282]:
clips=segments[0]

In [283]:
def mel_transform(sample_rate:float,audio:torch.Tensor,window_size: float=0.02,hop_size:float=0.005,n_mels:int=64)->torch.Tensor:
    """
    transform audio data into mel sepctrogram
    """
    # Determine window size and frame shift
    # window_size = 0.04 # 40 milliseconds
    # hop_size = 0.02 # 20 milliseconds, usually half the window size
    # n_fft = int(window_size * sample_rate) # Convert window size to number of sampling points
    n_fft=1024
    # hop_length defines the overlap between windows, affecting the resolution of the spectrum graph on the time axis
    # hop_length = int(hop_size * sample_rate) # Convert frame shift to sampling point number
    hop_length=160

    # Calculate Mel Spectrogram
    # n_mels = 40 # Number of Mel filters

    # Set up Mel Spectrogram converter
    mel_transformer = MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=640,
        n_mels=n_mels,
        f_min=0,
        f_max=16000
    )

    melspec=mel_transformer(audio)

    return melspec

In [284]:
clips = mel_transform(sample_rate=32000, audio=clips)

In [285]:
clips

tensor([[[4.3444e-09, 8.3298e-10, 3.3775e-09,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [3.3504e-09, 2.7876e-09, 1.2644e-08,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.3824e-09, 3.0708e-09, 3.1785e-08,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [2.0626e-06, 1.4593e-06, 4.4943e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.7839e-06, 2.4552e-06, 5.9570e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0122e-06, 1.6945e-06, 5.0962e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]])

In [286]:
print(clips.shape)

torch.Size([1, 64, 6001])


In [287]:
def compute_deltas(specgram: torch.Tensor, win_length: int = 5, mode: str = "replicate") -> torch.Tensor:
    """Compute delta coefficients of a tensor, usually a spectrogram.

    Args:
        specgram (Tensor): Tensor of audio of dimension (..., freq, time)
        win_length (int, optional): The window length used for computing delta (Default: 5)
        mode (str, optional): Mode parameter passed to padding (Default: "replicate")

    Returns:
        Tensor: Tensor of deltas of dimension (..., freq, time)
    """
    device = specgram.device  # Get the device of the input tensor
    dtype = specgram.dtype

    # pack batch
    shape = specgram.size()
    specgram = specgram.reshape(1, -1, shape[-1])

    assert win_length >= 3
    n = (win_length - 1) // 2
    denom = n * (n + 1) * (2 * n + 1) / 3

    specgram = torch.nn.functional.pad(specgram, (n, n), mode=mode)

    # Create the kernel tensor, making sure it is on the same device as the input tensor
    kernel = torch.arange(-n, n + 1, 1, dtype=dtype,device=device).repeat(specgram.shape[1], 1, 1)

    output = (
        torch.nn.functional.conv1d(specgram, kernel, groups=specgram.shape[1]) / denom
    )

    # unpack batch
    output = output.reshape(shape)

    return output



def make_delta(input_tensor: torch.Tensor):
    input_tensor = input_tensor.transpose(3, 2)
    input_tensor = compute_deltas(input_tensor)
    input_tensor = input_tensor.transpose(3, 2)
    return input_tensor


def image_delta(x):
    delta_1 = make_delta(x)
    delta_2 = make_delta(delta_1)
    x = torch.cat([x, delta_1, delta_2], dim=1)
    return x

In [288]:
# Because batch is not introduced yet, manually add clips to batch=1 and calculate delta and double-delta
clips=clips.unsqueeze(1)

print(clips.shape)

torch.Size([1, 1, 64, 6001])


In [289]:
clips=image_delta(x=clips)

In [290]:
print(clips.shape)

torch.Size([1, 3, 64, 6001])


In [291]:
# Load the pre-trained model
pretrained_model = timm.create_model('tf_efficientnetv2_s_in21k', pretrained=True,in_chans=3)

In [292]:
summary(pretrained_model,input_size=(1,3,64,6001))

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  [1, 21843]                --
├─Conv2dSame: 1-1                             [1, 24, 32, 3001]         648
├─BatchNormAct2d: 1-2                         [1, 24, 32, 3001]         48
│    └─Identity: 2-1                          [1, 24, 32, 3001]         --
│    └─SiLU: 2-2                              [1, 24, 32, 3001]         --
├─Sequential: 1-3                             [1, 256, 2, 188]          --
│    └─Sequential: 2-3                        [1, 24, 32, 3001]         --
│    │    └─ConvBnAct: 3-1                    [1, 24, 32, 3001]         5,232
│    │    └─ConvBnAct: 3-2                    [1, 24, 32, 3001]         5,232
│    └─Sequential: 2-4                        [1, 48, 16, 1501]         --
│    │    └─EdgeResidual: 3-3                 [1, 48, 16, 1501]         25,632
│    │    └─EdgeResidual: 3-4                 [1, 48, 16, 1501]         92,640
│    

In [293]:
def init_layer(layer):
    '''
    initilize dense layers' parameters
    '''
    nn.init.xavier_uniform_(layer.weight) # Initialize the weights and biases of the network layer

    if hasattr(layer, "bias"): # Check if the layer has a bias attribute
        if layer.bias is not None: # and bias is not None
            layer.bias.data.fill_(0.0) # If there is a bias, initialize it to 0

In [294]:
# Later we want to pass the acquired high-dimensional features into an attention module

class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla
        # x: This is the final output after the attention weights and classification layer.
        # shape: (n_samples, out_features). Since the time dimension is summed and compressed, each sample and each output feature ends up having a single value.
        # norm_att: This is the output of the attention layer (att) after the softmax and tanh functions, which shows which parts of the input sequence the model should focus on. 
        # Normalization ensures that the attention weights for all time steps add up to 1, which makes it easier to interpret the importance of each time step.
        # shape: (n_samples, out_features, n_time), where out_features is the number of output features of the att convolutional layer, which is the same as the out_features argument of the input. 
        # Each time step and each output feature has a normalized weight.
        # cla: This is the output of the classification layer (cla), which is obtained by processing the input features through another 1D convolutional layer. 
        # This output layer is often used to directly predict task-related outputs, such as the probability of a class label.
        # Shape: (n_samples, out_features, n_time), same shape as norm_att. This means that each output feature corresponding to each time step has a value processed by the activation function.
    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)

In [295]:
layers = list(pretrained_model.children())[:-2]

In [296]:
encoder = nn.Sequential(*layers)

In [298]:
clips=encoder(clips)

In [300]:
clips.shape

torch.Size([1, 1280, 2, 188])

In [302]:
clips = torch.mean(clips, dim=2)

In [303]:
clips.shape

torch.Size([1, 1280, 188])

In [304]:
x1 = F.max_pool1d(clips, kernel_size=3, stride=1, padding=1)
x2 = F.avg_pool1d(clips, kernel_size=3, stride=1, padding=1)

x=x1+x2

In [306]:
x.shape

torch.Size([1, 1280, 188])

In [307]:
x = x.transpose(1, 2)

fc1 = nn.Linear(in_features=1280, out_features=1280, bias=True)
x = F.relu_(fc1(x))

In [308]:
x.shape

torch.Size([1, 188, 1280])

In [309]:
x = x.transpose(1, 2)

x.shape

torch.Size([1, 1280, 188])

In [310]:
att_block=AttBlockV2(in_features=1280, out_features=182, activation="sigmoid")
target_pred, norm_att, segmentwise_output = att_block(x)

In [311]:
target_pred

tensor([[0.5608, 0.5782, 0.1967, 0.4402, 0.4739, 0.2244, 0.5200, 0.4540, 0.6897,
         0.4866, 0.4236, 0.4556, 0.3476, 0.6344, 0.4943, 0.3387, 0.3350, 0.6841,
         0.4972, 0.3843, 0.6362, 0.7239, 0.4768, 0.6738, 0.4576, 0.4403, 0.6815,
         0.4899, 0.4575, 0.4765, 0.3783, 0.4529, 0.6115, 0.5004, 0.3865, 0.6503,
         0.3329, 0.3666, 0.2296, 0.4565, 0.3741, 0.6746, 0.5814, 0.6070, 0.4672,
         0.4838, 0.5268, 0.6186, 0.7956, 0.4798, 0.7582, 0.2901, 0.4931, 0.6023,
         0.5570, 0.5558, 0.4440, 0.6446, 0.3110, 0.3769, 0.4136, 0.4235, 0.7147,
         0.5674, 0.5391, 0.3794, 0.7269, 0.4311, 0.5004, 0.5672, 0.6284, 0.5760,
         0.7861, 0.6082, 0.4633, 0.6593, 0.3987, 0.5142, 0.6563, 0.6528, 0.6056,
         0.2514, 0.3870, 0.3149, 0.4226, 0.3753, 0.3722, 0.6241, 0.5200, 0.5041,
         0.3596, 0.4487, 0.5331, 0.5930, 0.3398, 0.3865, 0.4588, 0.4081, 0.4368,
         0.5722, 0.3550, 0.5919, 0.5105, 0.3923, 0.4679, 0.4784, 0.5790, 0.5515,
         0.6007, 0.4830, 0.5

In [297]:
class BirdModelModule(L.LightningModule):

    def __init__(self,sample_rate:int=32000,pretrained_model_name:str='tf_efficientnetv2_s_in21k',class_num:int=182):
        super().__init__()
        self.sample_rate=sample_rate
        self.class_num=class_num

        # Load the pre-trained model
        pretrained_model = timm.create_model(pretrained_model_name, pretrained=True,in_chans=3)

        # The last two layers are an adaptive pooling layer and a fully connected layer
        # Here I choose to replace these two layers, first remove these two layers
        layers = list(pretrained_model.children())[:-2]

        self.encoder = nn.Sequential(*layers)

        self.in_features=pretrained_model.classifier.in_features # classifier is the last fully connected layer of the model, out_features represents the number of categories

        # Create a fully connected layer
        self.fc1 = nn.Linear(in_features=self.in_features, out_features=self.in_features, bias=True).to(device)

        # Add attention block
        self.att_block=AttBlockV2(in_features=self.in_features, out_features=self.class_num, activation="sigmoid").to(device)

        # Initialize the weights and biases of the fully connected layer
        init_layer(self.fc1)

        # loss function
        self.loss_function = nn.BCEWithLogitsLoss(reduction="none")


    def forward(self,clip):

        # Calculate the mean of each frequency band and merge them to compress the dimension
        clip = torch.mean(clip, dim=2)

        # channel smoothing
        x1 = F.max_pool1d(clip, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(clip, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.3, training=True)

        x = x.transpose(1, 2)

        x = F.relu_(self.fc1(x))

        x = x.transpose(1, 2)

        x = F.dropout(x, p=0.3, training=True)

        target_pred, norm_att, segmentwise_output = self.att_block(x)

        
        return target_pred




    def training_step(self,batch,batch_idx):


        pass


    def validation_step(self,batch,batch_idx):
        pass

        

    def configure_optimizers(self):
        pass

    def on_train_epoch_end(self):
        pass


    def on_validation_epoch_end(self):
        pass

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        pass